In [4]:
!pip install timm flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.2 MB/s eta 0:00:00
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn


In [2]:
!unzip res_data.zip

Archive:  res_data.zip
   creating: res_data/
  inflating: res_data/0.png          
  inflating: res_data/1.png          
  inflating: res_data/10.png         
  inflating: res_data/100.png        
  inflating: res_data/101.png        
  inflating: res_data/102.png        
  inflating: res_data/103.png        
  inflating: res_data/104.png        
  inflating: res_data/105.png        
  inflating: res_data/106.png        
  inflating: res_data/107.png        
  inflating: res_data/108.png        
  inflating: res_data/109.png        
  inflating: res_data/11.png         
  inflating: res_data/110.png        
  inflating: res_data/111.png        
  inflating: res_data/112.png        
  inflating: res_data/113.png        
  inflating: res_data/114.png        
  inflating: res_data/115.png        
  inflating: res_data/116.png        
  inflating: res_data/117.png        
  inflating: res_data/118.png        
  inflating: res_data/119.png        
  inflating: res_data/12.png         
  in

In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor, AutoConfig
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("thwri/CogFlorence-2.2-Large", trust_remote_code=True).to(device).eval()
processor = AutoProcessor.from_pretrained("thwri/CogFlorence-2.2-Large", trust_remote_code=True)
# Function to run the model on an example
def run_example(task_prompt, image):
    prompt = task_prompt
    # Ensure the image is in RGB mode
    if image.mode != "RGB":
        image = image.convert("RGB")
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3,
        do_sample=True
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    return parsed_answer
from PIL import Image
import requests
import copy


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
imgs_folder = "res_data"
import os
from tqdm import tqdm


all_imgs = [Image.open(os.path.join(imgs_folder, img_name)) for img_name in os.listdir(imgs_folder)]
desc = []
for img in tqdm(all_imgs):
    base_decription = run_example("<DETAILED_CAPTION>" , img)
    desc.append(base_decription)

100%|██████████| 470/470 [10:27<00:00,  1.33s/it]


In [19]:
c1 = os.listdir(imgs_folder)
c2 = [i["<DETAILED_CAPTION>"] for i in desc]
import pandas as pd

df = pd.DataFrame({"file_name": c1, "description": c2})
df


,file_name,description
0,358.png,The image shows a pair of black rain boots wit...
1,272.png,The image shows a man in a white shirt and bla...
2,488.png,The image shows a metal object with a dark bac...
3,24.png,The image shows a children's book with a vibra...
4,72.png,The image shows a white gold engagement ring w...
...,...,...
465,266.png,"The image shows a laptop, phone, and tablet di..."
466,422.png,The image shows a person holding a flower in t...
467,476.png,The image shows a pair of gold earrings with a...
468,87.png,The image shows a woman standing against a dar...


In [20]:
df.to_csv("descriptions.csv", index=False)

In [21]:
from google.colab import files
files.download('descriptions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>